# Drive  

In [ ]:
# https://drive.google.com/file/d/19MIw3ZI-Z91NLTDMc8GI4OlIOwN_Vv-t/view?usp=sharing
# gdown https://drive.google.com/uc?id=19MIw3ZI-Z91NLTDMc8GI4OlIOwN_Vv-t

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import statements

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import json
import datetime
import copy
from PIL import Image
from PIL import Image as im
import joblib
from sklearn.model_selection import train_test_split
# import math as Math
import random
import torch.optim

In [9]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision

In [10]:
import cv2
import joblib 

In [11]:
# !pip install scipy==1.1.0

In [12]:
import os
import numpy as np
import h5py
import json
import torch
from scipy.misc import imread, imresize
from tqdm import tqdm
from collections import Counter
from random import seed, choice, sample

In [13]:
 #Import libraries
import pandas as pd
import numpy as np
from google.colab import drive
import glob
import string
import nltk
import joblib
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
# #Reading Files
# drive.mount('/content/drive')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Saving and Loading functions 

In [14]:
# Saving and Loading models using joblib 
def save(filename, obj):
  with open(filename, 'wb') as handle:
      joblib.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
  with open(filename, 'rb') as handle:
      return joblib.load(filename)


# Paths  

In [15]:
# Final folder where all the data saved 
# output_folder = "/content/drive/MyDrive/SEM-2/05-DL /Assignments/A4/Method2/data/"
output_folder = "./"

# Data Downloading 

In [16]:
# # !wget https://drive.google.com/file/d/1lbqTV-u8xmZ3eBuQ4tUSjq0mOjaIIp_P 
# !unzip "/content/drive/MyDrive/SEM-2/05-DL /Assignments/A4/Data/Dataset.zip" -d /content/drive/MyDrive/SEM-2/05-DL /Assignments/A4/Data/

In [17]:
# !du -sh "/content/drive/MyDrive/A4/Data/"


In [18]:

# !unzip "/content/drive/MyDrive/SEM-2/05-DL /Assignments/A4/Data/Dataset.zip" -d "/content/drive/MyDrive/SEM-2/05-DL /Assignments/A4/Data/myData/"

# Data Preprocessing 

## Loading Data Files

In [19]:
# IITD
# p = "/content/drive/MyDrive/SEM-2/05-DL /Assignments/A4/"
# gmail 
p = "/content/drive/MyDrive/A4/"
# !ls "/content/drive/MyDrive/SEM-2/05-DL /Assignments/A4/Data/myData/Data/Val"

In [20]:
f1 = load(p + "Data/myData/Data/Val/val_captions.pkl")
f2 = load(p + "Data/myData/Data/Test/test_captions.pkl")
f3 = load(p + "Data/myData/Data/Train/train_captions.pkl")

In [21]:
# """
#     Creates input files for training, validation, and test data.
#     :param image_folder: folder with downloaded images
#     :param captions_per_image: number of captions to sample per image
#     :param min_word_freq: words occuring less frequently than this threshold are binned as <unk>s
#     :param output_folder: folder to save files
#     :param max_len: don't sample captions longer than this length
#     """

## word frequency counter 

In [22]:
def inputPreprocess(documentText):
  def remove_punc(word):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punc_list = [char if char not in punctuations else ' ' for char in word]
    return ''.join(no_punc_list)

  #Convert the text to lower case
  documentText = documentText.lower()
  
  #removing punctuations from the string
  no_punc_sentence = remove_punc(documentText)
  #documentText = documentText.translate(str.maketrans('','',string.punctuation))

  #Perform word tokenization
  listToken = word_tokenize(no_punc_sentence)

  # #removing stopwords from tokens
  # stopWords = set(stopwords.words('english'))  
  # filtered_sentence = [token for token in listToken if token not in stopWords]   


  # #remove blank space tokens
  # no_space_sentence = [token for token in filtered_sentence if token.strip()]   
  
  # #removing single char word and \x type chars
  # #filtered_sentence = [token for token in listToken if len(token) > 1]      
  
  return listToken

In [23]:
# calculates the total word frequency for all train,test and val 
word_freq = Counter()
for file in (f1,f2,f3):
  for k in file.keys():
    for sentence in file[k]:
      word_freq.update(inputPreprocess(sentence))

In [24]:
# word_freq

## Creating a word map 

In [26]:
# mapping each word to a particular index 
min_word_freq = 1
# Create word map
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 4 for v, k in enumerate(words)}
word_map['<start>'] = 0
word_map['<eos>'] = 1
word_map['<unk>'] = 2
word_map['<pad>'] = 3

word_map_reverse = {str(v + 4) : k for v, k in enumerate(words)}
word_map_reverse['0'] = '<start>'
word_map_reverse['1'] = '<eos>'
word_map_reverse['2'] = '<unk>'
word_map_reverse['3'] = '<pad>'

In [28]:
# word_map_reverse

## Train, Test, Val paths and captions 

In [40]:
# Read image paths and captions for each image
train_image_paths = []
train_image_captions = []
val_image_paths = []
val_image_captions = []
test_image_paths = []
test_image_captions = []

In [29]:
# Not giving absolute path but variable path or relative path here 
image_folder = "Data"
base_folder = ["Val", "Test","Train"] 

In [44]:
# With maximum length 50 we get 5 captions for each image in Train,Test,Val 
max_len = 50
for idx,file in enumerate((f1,f2,f3)):
  for k in file.keys():
    path = os.path.join(image_folder,base_folder[idx],'Images/'+ k)
    
    for sentence in file[k]:
      if len(inputPreprocess(sentence)) <= max_len:
        captions = inputPreprocess(sentence)

      # if len(captions) == 0:
      #       continue

        if idx == 2:
          train_image_paths.append(path)
          train_image_captions.append(captions)
        elif idx == 0:
          val_image_paths.append(path)
          val_image_captions.append(captions)
        elif idx == 1:
          test_image_paths.append(path)
          test_image_captions.append(captions)        


In [45]:
# Got paths of all images and their array of captions for each Train, Test and Val 
# check if lengths equal or not 
 # Sanity check
assert len(train_image_paths) == len(train_image_captions)
assert len(val_image_paths) == len(val_image_captions)
assert len(test_image_paths) == len(test_image_captions)

In [46]:
# remove half of the images and captions in training 
train_image_paths = train_image_paths[:10000]
train_image_captions = train_image_captions[:10000]

In [47]:
len(train_image_paths)

10000

In [48]:
train_image_paths[0]

'Data/Train/Images/2513260012_03d33305cf.jpg'

## Saving word map 

In [30]:
# # # Save word map to a JSON

# with open(output_folder + '/word_dict.json', 'w') as f:
#         json.dump(word_map, f)

save('./word_dict.json',word_map)
save('./word_dict_reverse.json',word_map_reverse)

## Saving Image, Captions, Caption lengths 

In [50]:
# Sample captions for each image, save images to HDF5 file, and captions and their lengths to JSON files
seed(123)

In [51]:
def process_caption_tokens(caption_tokens, word_dict, max_length):
    captions = []
    for tokens in caption_tokens:
        token_idxs = [word_dict[token] if token in word_dict else word_dict['<unk>'] for token in tokens]
        captions.append(
            [word_dict['<start>']] + token_idxs + [word_dict['<eos>']] +
            [word_dict['<pad>']] * (max_length - len(tokens)))

    return captions

In [52]:
max_length = 50
train_image_captions = process_caption_tokens(train_image_captions, word_map, max_length)
val_image_captions = process_caption_tokens(val_image_captions, word_map, max_length)
test_image_captions = process_caption_tokens(test_image_captions, word_map, max_length)

In [53]:
  # with open(output_folder + '/train_img_paths.json', 'w') as f:
  #     json.dump(train_image_paths, f)
  # with open(output_folder + '/val_img_paths.json', 'w') as f:
  #     json.dump(val_image_paths, f)
  # with open(output_folder + '/test_img_paths.json', 'w') as f:
  #    json.dump(test_image_paths, f)
  # with open(output_folder + '/train_captions.json', 'w') as f:
  #     json.dump(train_image_captions, f)
  # with open(output_folder + '/val_captions.json', 'w') as f:
  #     json.dump(val_image_captions, f)
  # with open(output_folder + '/test_captions.json', 'w') as f:
  #     json.dump(test_image_captions, f)

save('./train_img_paths.json',train_image_paths)
save('./val_img_paths.json',val_image_paths)
save('./test_img_paths.json',test_image_paths)
save('./train_captions.json',train_image_captions)
save('./val_captions.json',val_image_captions)
save('./test_captions.json',test_image_captions)

In [31]:
# downloading all preprocessed files from collab 
from google.colab import files
files.download('train_img_paths.json')
files.download('val_img_paths.json')
files.download('test_img_paths.json')
files.download('train_captions.json')
files.download('val_captions.json')
files.download('test_captions.json')
files.download('word_dict.json')
files.download('word_dict_reverse.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Preprocessed Files 

In [ ]:
# !ls "/content/drive/MyDrive/SEM-2/05-DL /Assignments/A4/Method2/data/"

test_captions.json   train_captions.json   val_captions.json   word_dict.json
test_img_paths.json  train_img_paths.json  val_img_paths.json


In [55]:
# !ls 

drive		    test_img_paths.json   val_captions.json
sample_data	    train_captions.json   val_img_paths.json
test_captions.json  train_img_paths.json  word_dict.json
